In [35]:

class Crop:
    def __init__(self, name, min_moisture, max_moisture):
        self.name = name
        self.min_moisture = min_moisture
        self.max_moisture = max_moisture

#Sample field data (you can later replace this with CSV reading)
fields = [
    {"FieldID": "F01", "Crop": "Maize", "Current": 28},
    {"FieldID": "F02", "Crop": "Tomatoes", "Current": 41},
    {"FieldID": "F03", "Crop": "Cabbage", "Current": 55}
]

#Define crop moisture ranges
crop_data = {
    "Maize": Crop("Maize", 30, 40),
    "Tomatoes": Crop("Tomatoes", 45, 55),
    "Cabbage": Crop("Cabbage", 50, 60)
}

#Check need and calculate need scores
need_list = []
for field in fields:
    crop = crop_data[field["Crop"]]
    current = field["Current"]
    if current < crop.min_moisture:
        need = crop.min_moisture - current
        need_list.append({
            "FieldID": field["FieldID"],
            "Crop": field["Crop"],
            "Need": need
        })
        
#Sort by highest need
need_list.sort(key=lambda x: x["Need"], reverse=True)

#Water allocation
budget = 5000
water_per_percent = 50
print("Irrigation Schedule:")
for field in need_list:
    max_possible = min(field["Need"], budget // water_per_percent)
    if max_possible > 0:
        water_used = max_possible * water_per_percent
        budget -= water_used
        print(f"{field['FieldID']} ({field['Crop']}): {max_possible}% -> {water_used} m³")

print(f"Remaining water: {budget} m³")

Irrigation Schedule:
F02 (Tomatoes): 4% -> 200 m³
F01 (Maize): 2% -> 100 m³
Remaining water: 4700 m³


In [27]:


import numpy as np  # Import NumPy library for working with arrays

# ---------------- STEP 1: Create 10x10 NDVI Array ----------------
# Initialize a 10x10 NumPy array with random float values between 0.2 and 0.9
# This simulates NDVI data captured by a drone
ndvi_data = np.random.uniform(0.2, 0.9, (10, 10))

# ---------------- STEP 2: Analyze Health Status ----------------
# NDVI Classification rules:
# Healthy: NDVI > 0.8
# Stressed: NDVI between 0.5 and 0.8 (inclusive)
# Bare Soil/Dead: NDVI < 0.5

# Create boolean masks for each category
healthy = ndvi_data > 0.8
stressed = (ndvi_data >= 0.5) & (ndvi_data <= 0.8)
bare_soil = ndvi_data < 0.5

# Count total number of cells (10x10 = 100)
total_cells = ndvi_data.size

# Calculate percentage for each category
healthy_percent = np.sum(healthy) / total_cells * 100
stressed_percent = np.sum(stressed) / total_cells * 100
bare_soil_percent = np.sum(bare_soil) / total_cells * 100

# ---------------- STEP 3: Find 2x2 Stressed Zones ----------------
# We look for 2x2 blocks where ALL 4 values are in the stressed range (0.5 to 0.8)
# Save the top-left corner coordinates of those blocks

stressed_zones = []  # List to store coordinates of stressed zones

# Loop through the array, stopping at row 8 and column 8 to form 2x2 blocks
for i in range(9):  # Rows 0 to 8
    for j in range(9):  # Columns 0 to 8
        # Get the 2x2 block starting at (i, j)
        block = ndvi_data[i:i+2, j:j+2]

        # Check if all values in the block are within stressed range
        if np.all((block >= 0.5) & (block <= 0.8)):
            # If true, save the top-left coordinate of the 2x2 block
            stressed_zones.append((i, j))

# ---------------- STEP 4: Print Summary Report ----------------
print("NDVI Health Analysis Report")
print("----------------------------")
print(f"Healthy vegetation: {healthy_percent:.2f}%")
print(f"Stressed vegetation: {stressed_percent:.2f}%")
print(f"Bare Soil/Dead: {bare_soil_percent:.2f}%")

# Print the coordinates of all detected 2x2 stressed zones
print("\n2x2 Stressed Zones (top-left corners):")
for zone in stressed_zones:
    print(f"  - Row: {zone[0]}, Column: {zone[1]}")



NDVI Health Analysis Report
----------------------------
Healthy vegetation: 11.00%
Stressed vegetation: 43.00%
Bare Soil/Dead: 46.00%

2x2 Stressed Zones (top-left corners):
  - Row: 2, Column: 8
  - Row: 3, Column: 8
  - Row: 6, Column: 5
  - Row: 7, Column: 5
  - Row: 7, Column: 6


In [9]:
from abc import ABC, abstractmethod

# Abstract base class
class Expense(ABC):
    @abstractmethod
    def get_cost(self):
        pass

# Variable expense per hectare
class VariableExpense(Expense):
    def __init__(self, item, cost_per_hectare):
        self.item = item
        self.cost_per_hectare = cost_per_hectare

    def get_cost(self):
        return self.cost_per_hectare

# Fixed expense (total cost)
class FixedExpense(Expense):
    def __init__(self, item, total_cost, area):
        self.item = item
        self.total_cost = total_cost
        self.area = area  # Total area in hectares

    def get_cost(self):
        return self.total_cost / self.area  # Per hectare

# CropEnterprise to represent each crop's profitability
class CropEnterprise:
    def __init__(self, name, yield_per_ha, price_per_tonne):
        self.name = name
        self.yield_per_ha = yield_per_ha
        self.price_per_tonne = price_per_tonne
        self.expenses = []

    def add_expense(self, expense):
        self.expenses.append(expense)

    def calculate_net_profit_per_hectare(self):
        revenue = self.yield_per_ha * self.price_per_tonne
        total_cost = sum(expense.get_cost() for expense in self.expenses)
        return revenue - total_cost
    def forecast_profit(self, price_increase_percent):
        new_price = self.price_per_tonne * (1 + price_increase_percent / 100)
        revenue = self.yield_per_ha * new_price
        total_cost = sum(expense.get_cost() for expense in self.expenses)
        return revenue - total_cost

# Main program
def main():
    crops = []

    for i in range(2):
        print(f"\nEnter details for Crop {i+1}:")
        name = input("Crop name: ")
        yield_per_ha = float(input("Expected yield (tonnes/ha): "))
        price_per_tonne = float(input("Market price (TZS/tonne): "))

        crop = CropEnterprise(name, yield_per_ha, price_per_tonne)

        area = float(input("Enter area under cultivation (hectares): "))

        n_var = int(input("Number of variable expenses: "))
        for _ in range(n_var):
            item = input("Variable item name: ")
            cost = float(input("Cost per hectare: "))
            crop.add_expense(VariableExpense(item, cost))
        
        n_fixed = int(input("Number of fixed expenses: "))
        for _ in range(n_fixed):
            item = input("Fixed item name: ")
            total_cost = float(input("Total cost: "))
            crop.add_expense(FixedExpense(item, total_cost, area))

        crops.append((crop, area))

    print("\n--- Net Profit Summary ---")
    total_current_profit = 0
    for crop, area in crops:
        net_profit = crop.calculate_net_profit_per_hectare()
        total_profit = net_profit * area
        total_current_profit += total_profit
        print(f"{crop.name}: Net profit/ha = TZS {net_profit:,.2f}, Total = TZS {total_profit:,.2f}")

    percent_increase = float(input("\nEnter % increase in market price to forecast future earnings: "))
    print("\n--- Forecasted Profit Summary ---")
    total_forecast_profit = 0
    for crop, area in crops:
        forecast_profit = crop.forecast_profit(percent_increase)
        total = forecast_profit * area
        total_forecast_profit += total
        print(f"{crop.name}: Forecasted net profit/ha = TZS {forecast_profit:,.2f}, Total = TZS {total:,.2f}")

    print("\n=== Summary ===")
    print(f"Current total farm profit: TZS {total_current_profit:,.2f}")
    print(f"Forecasted total farm profit after {percent_increase}% price increase: TZS {total_forecast_profit:,.2f}")

if __name__ == "__main__":
    main()






Enter details for Crop 1:


Crop name:  Maize
Expected yield (tonnes/ha):  3
Market price (TZS/tonne):  500000
Enter area under cultivation (hectares):  5
Number of variable expenses:  2
Variable item name:  Fertilizer
Cost per hectare:  120000
Variable item name:  Pesticide
Cost per hectare:  50000
Number of fixed expenses:  1
Fixed item name:  Tractor
Total cost:  1000000



Enter details for Crop 2:


Crop name:  Beans
Expected yield (tonnes/ha):  1
Market price (TZS/tonne):  300000
Enter area under cultivation (hectares):  5
Number of variable expenses:  2
Variable item name:  Fertilizer
Cost per hectare:  100000
Variable item name:  Pesticide
Cost per hectare:  50000
Number of fixed expenses:  1
Fixed item name:  Tractor
Total cost:  1000000



--- Net Profit Summary ---
Maize: Net profit/ha = TZS 1,130,000.00, Total = TZS 5,650,000.00
Beans: Net profit/ha = TZS -50,000.00, Total = TZS -250,000.00



Enter % increase in market price to forecast future earnings:  15



--- Forecasted Profit Summary ---
Maize: Forecasted net profit/ha = TZS 1,355,000.00, Total = TZS 6,775,000.00
Beans: Forecasted net profit/ha = TZS -5,000.00, Total = TZS -25,000.00

=== Summary ===
Current total farm profit: TZS 5,400,000.00
Forecasted total farm profit after 15.0% price increase: TZS 6,750,000.00


In [37]:
from abc import ABC, abstractmethod

# Define an abstract base class for all expenses
class Expense(ABC):
    @abstractmethod
    def get_cost(self):
        # Abstract method to be implemented by all subclasses
        pass

# Represents a variable expense (e.g., fertilizer, pesticide) that is charged per hectare
class VariableExpense(Expense):
    def __init__(self, item, cost_per_hectare):
        self.item = item  # Name of the expense
        self.cost_per_hectare = cost_per_hectare  # Cost in TZS per hectare

    def get_cost(self):
        return self.cost_per_hectare

# Represents a fixed expense (e.g., equipment, storage) with a total cost shared over area
class FixedExpense(Expense):
    def __init__(self, item, total_cost, area):
        self.item = item  # Name of the expense
        self.total_cost = total_cost  # Total cost for the item
        self.area = area  # Area over which the cost is spread

    def get_cost(self):
        # Return cost per hectare by dividing total cost by area
        return self.total_cost / self.area

# Represents a crop enterprise with yield, market price, and associated expenses
class CropEnterprise:
    def __init__(self, name, yield_per_ha, price_per_tonne):
        self.name = name  # Crop name (e.g., maize)
        self.yield_per_ha = yield_per_ha  # Yield in tonnes per hectare
        self.price_per_tonne = price_per_tonne  # Market price in TZS per tonne
        self.expenses = []  # List to hold Variable and Fixed Expense objects

    def add_expense(self, expense):
        # Add an expense to the crop's list
        self.expenses.append(expense)

    def calculate_net_profit_per_hectare(self):
        # Revenue = yield × price
        revenue = self.yield_per_ha * self.price_per_tonne
        # Total cost = sum of all expense costs per hectare
        total_cost = sum(expense.get_cost() for expense in self.expenses)
        # Net profit = revenue - total cost
        return revenue - total_cost

    def forecast_profit(self, price_increase_percent):
        # Increase price by a percentage for forecasting
        new_price = self.price_per_tonne * (1 + price_increase_percent / 100)
        revenue = self.yield_per_ha * new_price
        total_cost = sum(expense.get_cost() for expense in self.expenses)
        return revenue - total_cost

# Main function that drives the budgeting program
def main():
    crops = []  # List to store crop objects and their areas

    # Loop to model two different crops
    for i in range(2):
        print(f"\nEnter details for Crop {i+1}:")
        name = input("Crop name: ")
        yield_per_ha = float(input("Expected yield (tonnes/ha): "))
        price_per_tonne = float(input("Market price (TZS/tonne): "))

        # Create a CropEnterprise object
        crop = CropEnterprise(name, yield_per_ha, price_per_tonne)

        # Get area under cultivation for this crop
        area = float(input("Enter area under cultivation (hectares): "))

        # Add variable expenses
        n_var = int(input("Number of variable expenses: "))
        for _ in range(n_var):
            item = input("Variable item name: ")
            cost = float(input("Cost per hectare: "))
            crop.add_expense(VariableExpense(item, cost))

        # Add fixed expenses
        n_fixed = int(input("Number of fixed expenses: "))
        for _ in range(n_fixed):
            item = input("Fixed item name: ")
            total_cost = float(input("Total cost: "))
            crop.add_expense(FixedExpense(item, total_cost, area))

        # Store the crop and its area for profit calculations
        crops.append((crop, area))

    # Calculate and display current net profit for each crop
    print("\n--- Net Profit Summary ---")
    total_current_profit = 0
    for crop, area in crops:
        net_profit = crop.calculate_net_profit_per_hectare()
        total_profit = net_profit * area
        total_current_profit += total_profit
        print(f"{crop.name}: Net profit/ha = TZS {net_profit:,.2f}, Total = TZS {total_profit:,.2f}")

    # Get forecast percentage increase from user
    percent_increase = float(input("\nEnter % increase in market price to forecast future earnings: "))

    # Display forecasted profit with increased prices
    print("\n--- Forecasted Profit Summary ---")
    total_forecast_profit = 0
    for crop, area in crops:
        forecast_profit = crop.forecast_profit(percent_increase)
        total = forecast_profit * area
        total_forecast_profit += total
        print(f"{crop.name}: Forecasted net profit/ha = TZS {forecast_profit:,.2f}, Total = TZS {total:,.2f}")

    # Final summary output
    print("\n=== Summary ===")
    print(f"Current total farm profit: TZS {total_current_profit:,.2f}")
    print(f"Forecasted total farm profit after {percent_increase}% price increase: TZS {total_forecast_profit:,.2f}")

# Run the main function
if __name__ == "__main__":
    main()

"""
Assuming Maize (crop 1)

    Yield: 3 t/ha

    Market Price: 500,000 TZS/t

    Area: 5 ha

    Variable: Fertilizer (120k), Pesticide (50k)

    Fixed: Tractor (1,000,000 TZS)
    """

"""
 Crop 2: Beans

    Expected yield (tonnes/ha): 1.8

    Market price (TZS/tonne): 750000

    Area under cultivation (hectares): 3

Variable Expenses (per hectare):

    Seeds — 60000 TZS

    Fertilizer — 85000 TZS

    Labor — 40000 TZS

Fixed Expenses (total cost):

    Irrigation pump maintenance — 300000 TZS

    Storage facility — 600000 TZS
    """


Enter details for Crop 1:


Crop name:  maize
Expected yield (tonnes/ha):  3
Market price (TZS/tonne):  500000
Enter area under cultivation (hectares):  5
Number of variable expenses:  2
Variable item name:  fertilizer
Cost per hectare:  120000
Variable item name:  pesticide
Cost per hectare:  50000
Number of fixed expenses:  1
Fixed item name:  tractor
Total cost:  1000000



Enter details for Crop 2:


Crop name:  beans
Expected yield (tonnes/ha):  1.8
Market price (TZS/tonne):  750000
Enter area under cultivation (hectares):  3
Number of variable expenses:  3
Variable item name:  seeds
Cost per hectare:  60000
Variable item name:  fertilizer
Cost per hectare:  85000
Variable item name:  labor
Cost per hectare:  40000
Number of fixed expenses:  2
Fixed item name:  irrigation pump maintainance
Total cost:  300000
Fixed item name:  storage facility
Total cost:  600000



--- Net Profit Summary ---
maize: Net profit/ha = TZS 1,130,000.00, Total = TZS 5,650,000.00
beans: Net profit/ha = TZS 865,000.00, Total = TZS 2,595,000.00



Enter % increase in market price to forecast future earnings:  15



--- Forecasted Profit Summary ---
maize: Forecasted net profit/ha = TZS 1,355,000.00, Total = TZS 6,775,000.00
beans: Forecasted net profit/ha = TZS 1,067,500.00, Total = TZS 3,202,500.00

=== Summary ===
Current total farm profit: TZS 8,245,000.00
Forecasted total farm profit after 15.0% price increase: TZS 9,977,500.00


'\n Crop 2: Beans\n\n    Expected yield (tonnes/ha): 1.8\n\n    Market price (TZS/tonne): 750000\n\n    Area under cultivation (hectares): 3\n\nVariable Expenses (per hectare):\n\n    Seeds — 60000 TZS\n\n    Fertilizer — 85000 TZS\n\n    Labor — 40000 TZS\n\nFixed Expenses (total cost):\n\n    Irrigation pump maintenance — 300000 TZS\n\n    Storage facility — 600000 TZS\n    '

In [41]:
import random

class Cattle:
    def __init__(self, animal_id, gender, milk_yield, sire_id=None, dam_id=None):
        self.animal_id = animal_id
        self.gender = gender
        self.milk_yield = milk_yield
        self.sire_id = sire_id
        self.dam_id = dam_id

def produce_offspring(dam, sire, new_id):
    avg_yield =  ((dam.milk_yield + sire.milk_yield)/2)
    drift = avg_yield * random.uniform(-0.05, 0.05)
    return Cattle(new_id, random.choice(['male', 'female']), max(0, avg_yield + drift), sire.animal_id, dam.animal_id)

def get_avg_yield(herd):
    return sum(c.milk_yield for c in herd) / len(herd) if herd else 0

# Initial population
herd = [
    Cattle(1, 'male', 25), Cattle(2, 'male', 28),
    Cattle(3, 'female', 30), Cattle(4, 'female', 32), Cattle(5, 'female', 29),
    Cattle(6, 'female', 35), Cattle(7, 'female', 31)
]
animal_id_counter = len(herd) + 1
initial_avg_yield = get_avg_yield(herd)
print(f"Starting Average Milk Yield: {initial_avg_yield:.2f} L/day")

# Simulate 5 generations
for gen in range(1, 6):
    bulls = sorted([c for c in herd if c.gender == 'male'], key=lambda x: x.milk_yield, reverse=True)[:2]
    cows = sorted([c for c in herd if c.gender == 'female'], key=lambda x: x.milk_yield, reverse=True)[:5]
    
    if not bulls or not cows: break

    new_offspring = []
    for cow in cows:
        new_offspring.append(produce_offspring(cow, random.choice(bulls), animal_id_counter))
        animal_id_counter += 1
    herd.extend(new_offspring)

# Results
final_avg_yield = get_avg_yield(herd)
print(f"Final Average Milk Yield: {final_avg_yield:.2f} L/day")
print(f"Improvement: {final_avg_yield - initial_avg_yield:.2f} L/day ({((final_avg_yield - initial_avg_yield) / initial_avg_yield) * 100:.2f}%)")

Starting Average Milk Yield: 30.00 L/day
Final Average Milk Yield: 30.52 L/day
Improvement: 0.52 L/day (1.72%)


In [49]:
# Constants
GRID_SIZE = 15
INFESTED = 'X'
HEALTHY = 'O'

def create_field():
    # Create a 15x15 field initialized with 'O' (healthy)
    return [[HEALTHY for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

def display_field(field):
    # Nicely display the field row by row
    for row in field:
        print(' '.join(row))
    print()

def spread_pest(field):
    # Track new infestations separately to avoid immediate spreading
    new_field = [row.copy() for row in field]
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if field[i][j] == INFESTED:
                # Infect adjacent (non-diagonal) healthy cells
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    ni, nj = i + dx, j + dy
                    if 0 <= ni < GRID_SIZE and 0 <= nj < GRID_SIZE and field[ni][nj] == HEALTHY:
                        new_field[ni][nj] = INFESTED
    return new_field
def apply_spot_treatment(field, row, col):
    # Treat the selected cell and its adjacent ones (non-diagonal) by turning them back to 'O'
    for dx, dy in [(0, 0), (-1, 0), (1, 0), (0, -1), (0, 1)]:
        ni, nj = row + dx, col + dy
        if 0 <= ni < GRID_SIZE and 0 <= nj < GRID_SIZE:
            field[ni][nj] = HEALTHY

def calculate_infestation(field):
    total = GRID_SIZE * GRID_SIZE
    infested = sum(cell == INFESTED for row in field for cell in row)
    return infested, infested / total * 100

def main():
    field = create_field()
    field[7][7] = INFESTED  # Starting infestation
    print("Day 0 - Initial Infestation:")
    display_field(field)

    for day in range(1, 11):
        field = spread_pest(field)
        print(f"Day {day}:")
        display_field(field)

        # After Day 5, ask user to apply spot treatment
        if day == 5:
            try:
                row = int(input("Enter row for spot treatment (0–14): "))
                col = int(input("Enter column for spot treatment (0–14): "))
                apply_spot_treatment(field, row, col)
                print(f"\nSpot treatment applied at ({row}, {col})")
                display_field(field)
            except ValueError:
                print("Invalid input. Skipping spot treatment.")

    # Final result
    print("Final Field State After 10 Days:")
    display_field(field)
    infested_count, percentage = calculate_infestation(field)
    print(f"Infested cells: {infested_count} / {GRID_SIZE**2}")
    print(f"Infestation Percentage: {percentage:.2f}%")

if __name__ == "__main__":
    main()

Day 0 - Initial Infestation:
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O X O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O

Day 1:
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O X O O O O O O O
O O O O O O X X X O O O O O O
O O O O O O O X O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O

Day 2:
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O

Enter row for spot treatment (0–14):  7
Enter column for spot treatment (0–14):  7



Spot treatment applied at (7, 7)
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O X O O O O O O O
O O O O O O X X X O O O O O O
O O O O O X X X X X O O O O O
O O O O X X X X X X X O O O O
O O O X X X X O X X X X O O O
O O X X X X O O O X X X X O O
O O O X X X X O X X X X O O O
O O O O X X X X X X X O O O O
O O O O O X X X X X O O O O O
O O O O O O X X X O O O O O O
O O O O O O O X O O O O O O O
O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O

Day 6:
O O O O O O O O O O O O O O O
O O O O O O O X O O O O O O O
O O O O O O X X X O O O O O O
O O O O O X X X X X O O O O O
O O O O X X X X X X X O O O O
O O O X X X X X X X X X O O O
O O X X X X X X X X X X X O O
O X X X X X X O X X X X X X O
O O X X X X X X X X X X X O O
O O O X X X X X X X X X O O O
O O O O X X X X X X X O O O O
O O O O O X X X X X O O O O O
O O O O O O X X X O O O O O O
O O O O O O O X O O O O O O O
O O O O O O O O O O O O O O O

Day 7:
O O O O O O O X O O O O O O O
O O O O O O X X X O 

In [57]:
import math

# Define a class for collection centers and markets
class Location:
    def __init__(self, name, x, y, quantity):
        self.name = name
        self.x = x
        self.y = y
        self.quantity = quantity  # supply for centers, demand for markets

    def distance_to(self, other):
        # Euclidean distance between self and another Location
        return math.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)

def greedy_delivery_plan(collection_centers, markets):
    deliveries = []  # list to hold (from, to, tonnes, distance)
    total_distance = 0 
    
    for market in markets.values():
        while market.quantity > 0:
            # Find the closest center with supply > 0
            available_centers = [
                center for center in collection_centers.values() if center.quantity > 0
            ]

            if not available_centers:
                print(f" Cannot fully supply {market.name}, supply exhausted.")
                break

            # Sort by distance to market
            closest_center = min(available_centers, key=lambda c: c.distance_to(market))
            distance = closest_center.distance_to(market)
            supply_amount = min(closest_center.quantity, market.quantity)

            # Record the delivery
            deliveries.append((closest_center.name, market.name, supply_amount, distance))
            total_distance += distance

            # Update states
            closest_center.quantity -= supply_amount
            market.quantity -= supply_amount 
            
        return deliveries, total_distance

def main():
    # Define collection centers
    collection_centers = {
        "CC1": Location("CC1", 5, 10, 20),
        "CC2": Location("CC2", 10, 15, 25)
    }

    # Define markets
    markets = {
        "MarketA": Location("MarketA", 20, 30, 15),
        "MarketB": Location("MarketB", 25, 25, 20),
        "MarketC": Location("MarketC", 30, 10, 10)
    }

    # Run delivery planner
    deliveries, total_distance = greedy_delivery_plan(collection_centers, markets)

    # Print plan
    print("\n Delivery Plan:")
    for from_center, to_market, tonnes, dist in deliveries:
        print(f"{from_center} sends {tonnes} tonnes to {to_market} (Distance: {dist:.2f})")

    print(f"\n Total Distance Traveled: {total_distance:.2f} units")

if __name__ == "__main__":
    main()


 Delivery Plan:
CC2 sends 15 tonnes to MarketA (Distance: 18.03)

 Total Distance Traveled: 18.03 units


In [65]:
import random

# Device classes
class Heater:
    def __init__(self):
        self.on = False

    def turn_on(self):
        if not self.on:
            self.on = True
            print(" Heater turned ON.")

    def turn_off(self):
        if self.on:
            self.on = False
            print(" Heater turned OFF.")

class Fan:
    def __init__(self):
        self.on = False

    def turn_on(self):
        if not self.on:
            self.on = True
            print("Fan turned ON.")

    def turn_off(self):
        if self.on:
            self.on = False
            print(" Fan turned OFF.")

class Mister:
    def __init__(self):
        self.on = False

    def turn_on(self):
        if not self.on:
            self.on = True
            print(" Mister turned ON.")

    def turn_off(self):
        if self.on:
            self.on = False
            print(" Mister turned OFF.")

# Greenhouse with temperature and humidity
class Greenhouse:
    def __init__(self, temperature, humidity):
        self.temperature = temperature
        self.humidity = humidity 
        
    def update_conditions(self):
        # Simulate small random fluctuations
        self.temperature += random.uniform(-1.5, 1.5)
        self.humidity += random.uniform(-5, 5)
        # Clamp values to realistic bounds
        self.temperature = max(10, min(40, self.temperature))
        self.humidity = max(30, min(90, self.humidity))

# Controller that regulates climate
class ClimateController:
    def __init__(self, greenhouse, heater, fan, mister):
        self.greenhouse = greenhouse
        self.heater = heater
        self.fan = fan
        self.mister = mister

    def check_and_regulate(self):
        temp = self.greenhouse.temperature
        hum = self.greenhouse.humidity
        print(f" Temp: {temp:.1f}°C, Humidity: {hum:.1f}%")

        # Heater: turn on if too cold
        if temp < 21:
            self.heater.turn_on()
        else:self.heater.turn_off()

        # Fan: turn on if too hot
        if temp > 25:
            self.fan.turn_on()
        else:
            self.fan.turn_off()

        # Mister: turn on if humidity too low
        if hum < 60:
            self.mister.turn_on()
        else:
            self.mister.turn_off()

def run_simulation():
    # Initial values
    greenhouse = Greenhouse(temperature=22.5, humidity=65.0)
    heater = Heater()
    fan = Fan()
    mister = Mister()

    controller = ClimateController(greenhouse, heater, fan, mister)

    print("Starting 12-hour Greenhouse Climate Simulation \n")
    for hour in range(1, 13):
        print(f"\n--- Hour {hour} ---")
        greenhouse.update_conditions()
        controller.check_and_regulate()

if __name__ == "__main__":
    run_simulation()

Starting 12-hour Greenhouse Climate Simulation 


--- Hour 1 ---
 Temp: 21.9°C, Humidity: 63.9%

--- Hour 2 ---
 Temp: 21.2°C, Humidity: 63.9%

--- Hour 3 ---
 Temp: 22.6°C, Humidity: 59.0%
 Mister turned ON.

--- Hour 4 ---
 Temp: 21.3°C, Humidity: 58.7%

--- Hour 5 ---
 Temp: 20.4°C, Humidity: 53.9%
 Heater turned ON.

--- Hour 6 ---
 Temp: 19.3°C, Humidity: 53.7%

--- Hour 7 ---
 Temp: 20.0°C, Humidity: 55.4%

--- Hour 8 ---
 Temp: 18.5°C, Humidity: 54.4%

--- Hour 9 ---
 Temp: 17.2°C, Humidity: 52.0%

--- Hour 10 ---
 Temp: 16.1°C, Humidity: 52.4%

--- Hour 11 ---
 Temp: 17.0°C, Humidity: 52.2%

--- Hour 12 ---
 Temp: 16.6°C, Humidity: 53.1%


In [87]:
import json

class CropProfile:
    def __init__(self, name, n, p, k):
        self.name = name
        self.n_opt, self.p_opt, self.k_opt = n, p, k

def get_fertilizer_rec(soil_file, crop_name):
    """Generates fertilizer recommendations based on soil data and crop needs."""
    try:
        with open(soil_file, 'r') as f:
            soil_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error reading {soil_file}: {e}"); return

    crops = {
        "Maize": CropProfile("Maize", 50, 40, 100),
        "Wheat": CropProfile("Wheat", 45, 30, 80),
        "Soybeans": CropProfile("Soybeans", 30, 35, 90)
    }

    if crop_name not in crops:
        print(f"Crop '{crop_name}' not found. Available: {', '.join(crops.keys())}"); return

    crop = crops[crop_name]
    results = soil_data.get("results", {})
    actual_n = results.get("nitrogen_ppm", 0)
    actual_p = results.get("phosphorus_ppm", 0)
    actual_k = results.get("potassium_ppm", 0)

    # Calculate deficit (only if needed)
    n_deficit = max(0, crop.n_opt - actual_n)
    p_deficit = max(0, crop.p_opt - actual_p)
    k_deficit = max(0, crop.k_opt - actual_k)

    # Convert deficit (ppm) to kg/ha (1 ppm = 2.5 kg/ha fertilizer)
    KG_PER_HA_PER_PPM = 2.5
    rec_n = n_deficit * KG_PER_HA_PER_PPM
    rec_p = p_deficit * KG_PER_HA_PER_PPM
    rec_k = k_deficit * KG_PER_HA_PER_PPM

    print(f"\n--- Fertilizer Recommendation for {crop.name} ({soil_data.get('field_id', 'N/A')}) ---")
    print(f"Soil N: {actual_n}ppm (Optimal: {crop.n_opt}ppm) -> Deficit: {n_deficit}ppm")
    print(f"Soil P: {actual_p}ppm (Optimal: {crop.p_opt}ppm) -> Deficit: {p_deficit}ppm")
    print(f"Soil K: {actual_k}ppm (Optimal: {crop.k_opt}ppm) -> Deficit: {k_deficit}ppm")
    print(f"\nRecommendation: Apply {rec_n:.1f} kg/ha Nitrogen, {rec_p:.1f} kg/ha Phosphorus, {rec_k:.1f} kg/ha Potassium.")

# --- Main execution ---
if __name__ == "__main__":
    # Create dummy soil_data.json
    dummy_data = {"field_id": "F04-West", "test_date": "2025-06-10", "results": {"ph": 5.8, "nitrogen_ppm": 20, "phosphorus_ppm": 35, "potassium_ppm": 80}}
    with open('soil_data.json', 'w') as f: json.dump(dummy_data, f, indent=4)

    get_fertilizer_rec('soil_data.json', 'Maize')

    dummy_data_2 = {"field_id": "F01-East", "test_date": "2025-06-15", "results": {"ph": 6.5, "nitrogen_ppm": 60, "phosphorus_ppm": 30, "potassium_ppm": 50}}
    with open('soil_data_2.json', 'w') as f: json.dump(dummy_data_2, f, indent=4)

    print("\n" + "="*40 + "\n")
    get_fertilizer_rec('soil_data_2.json', 'Maize')

    print("\n" + "="*40 + "\n")
    get_fertilizer_rec('soil_data_2.json', 'Wheat')


--- Fertilizer Recommendation for Maize (F04-West) ---
Soil N: 20ppm (Optimal: 50ppm) -> Deficit: 30ppm
Soil P: 35ppm (Optimal: 40ppm) -> Deficit: 5ppm
Soil K: 80ppm (Optimal: 100ppm) -> Deficit: 20ppm

Recommendation: Apply 75.0 kg/ha Nitrogen, 12.5 kg/ha Phosphorus, 50.0 kg/ha Potassium.



--- Fertilizer Recommendation for Maize (F01-East) ---
Soil N: 60ppm (Optimal: 50ppm) -> Deficit: 0ppm
Soil P: 30ppm (Optimal: 40ppm) -> Deficit: 10ppm
Soil K: 50ppm (Optimal: 100ppm) -> Deficit: 50ppm

Recommendation: Apply 0.0 kg/ha Nitrogen, 25.0 kg/ha Phosphorus, 125.0 kg/ha Potassium.



--- Fertilizer Recommendation for Wheat (F01-East) ---
Soil N: 60ppm (Optimal: 45ppm) -> Deficit: 0ppm
Soil P: 30ppm (Optimal: 30ppm) -> Deficit: 0ppm
Soil K: 50ppm (Optimal: 80ppm) -> Deficit: 30ppm

Recommendation: Apply 0.0 kg/ha Nitrogen, 0.0 kg/ha Phosphorus, 75.0 kg/ha Potassium.


In [85]:
import os

class Machine:
    def __init__(self, mid, intervals):
        self.id = mid
        self.total_hours = 0
        self.intervals = intervals
        self.due_tasks = {}

    def add_hours(self, hours):
        self.total_hours += hours

    def check_maintenance(self):
        self.due_tasks = {}
        for task, interval in self.intervals.items():
            if self.total_hours >= interval:
                # Check if it has passed a multiple of the interval
                # Example: total_hours 105, interval 100. (105 // 100) * 100 = 100. 105 >= 100
                # Example: total_hours 205, interval 100. (205 // 100) * 100 = 200. 205 >= 200
                if self.total_hours // interval > (self.total_hours - self.intervals[task]) // interval:
                     self.due_tasks[task] = f"Due (at {self.total_hours:.1f}h, interval {interval}h)"
                elif self.total_hours % interval == 0: # Exactly on the interval
                     self.due_tasks[task] = f"Due (at {self.total_hours:.1f}h, exactly {interval}h interval)"


def process_logs(log_dir, maintenance_config):
    report = []
    if not os.path.isdir(log_dir):
        print(f"Error: Directory '{log_dir}' not found.")
        return report

    for fname in os.listdir(log_dir):
        if fname.startswith('tractor_T') and fname.endswith('.txt'):
            mid = fname.replace('tractor_', '').replace('.txt', '')
            machine = Machine(mid, maintenance_config)
            
            with open(os.path.join(log_dir, fname), 'r') as f:
                for line in f:
                    try:
                        hours = float(line.strip().split(',')[1])
                        machine.add_hours(hours)
                    except (ValueError, IndexError):
                        print(f"Warning: Skipping bad line in {fname}: {line.strip()}")
            
            machine.check_maintenance()
            for task, status in machine.due_tasks.items():
                report.append({'ID': machine.id, 'Task': task, 'Hours': machine.total_hours, 'Status': status})
    return report

def main():
    log_dir = 'tractor_logs'
    maintenance_config = {'Oil Change': 100, 'Filter Change': 200, 'Engine Check': 500}

    # Create dummy logs (same as previous, simplified for brevity here)
    os.makedirs(log_dir, exist_ok=True)
    with open(os.path.join(log_dir, 'tractor_T101.txt'), 'w') as f: f.write("2025-06-01,8.5\n2025-06-02,95.0\n") # Total 103.5
    with open(os.path.join(log_dir, 'tractor_T102.txt'), 'w') as f: f.write("2025-06-01,190.0\n2025-06-02,15.0\n") # Total 205.0

    maintenance_report = process_logs(log_dir, maintenance_config)

    print("\n--- Maintenance Report ---")
    if not maintenance_report:
        print("No maintenance due.")
    else:
        maintenance_report.sort(key=lambda x: (x['ID'], x['Task']))
        for job in maintenance_report:
            print(f"Tractor {job['ID']}: {job['Task']} - {job['Status']}")

if __name__ == "__main__":
    main()





--- Maintenance Report ---
Tractor T101: Oil Change - Due (at 103.5h, interval 100h)
Tractor T102: Filter Change - Due (at 205.0h, interval 200h)
Tractor T102: Oil Change - Due (at 205.0h, interval 100h)


In [79]:
class Field:
    def __init__(self, initial_n=50, critical_n=25):
        self.nitrogen = initial_n
        self.critical = critical_n

    def update_n(self, effect):
        self.nitrogen += effect
        print(f"Nitrogen: {self.nitrogen} units.")
        if self.nitrogen < self.critical:
            print("WARNING: Nitrogen critically low!")

def simulate_rotation():
    field = Field()
    crop_effects = {'Maize': -20, 'Wheat': -18, 'Beans': +15, 'Peas': +12, 'Soybeans': +10, 'Fallow': +5, 'Clover': +20}

    print("--- Crop Rotation Planner ---")
    print(f"Initial Nitrogen: {field.nitrogen} units. Critical: {field.critical} units.")
    print("Crops:", ', '.join(crop_effects.keys()))

    for year in range(1, 6):
        while True:
            choice = input(f"Year {year} crop ({'/'.join(crop_effects.keys())}): ").strip().title()
            if choice in crop_effects:
                field.update_n(crop_effects[choice])
                break
            else:
                print("Invalid crop. Try again.")

    print("\n--- Rotation Complete ---")
    print(f"Final Nitrogen: {field.nitrogen} units.")
    print("Soil Health: Good" if field.nitrogen >= field.critical else "Soil Health: Poor")

if __name__ == "__main__":
    simulate_rotation()

--- Crop Rotation Planner ---
Initial Nitrogen: 50 units. Critical: 25 units.
Crops: Maize, Wheat, Beans, Peas, Soybeans, Fallow, Clover


Year 1 crop (Maize/Wheat/Beans/Peas/Soybeans/Fallow/Clover):  Maize


Nitrogen: 30 units.


Year 2 crop (Maize/Wheat/Beans/Peas/Soybeans/Fallow/Clover):  Wheat


Nitrogen: 12 units.


...

Nitrogen: 27 units.


Year 4 crop (Maize/Wheat/Beans/Peas/Soybeans/Fallow/Clover):  Peas


Nitrogen: 39 units.


Year 5 crop (Maize/Wheat/Beans/Peas/Soybeans/Fallow/Clover):  Soybeans


Nitrogen: 49 units.

--- Rotation Complete ---
Final Nitrogen: 49 units.
Soil Health: Good


In [91]:
P = 50000000
annual_rate = 0.12
r = annual_rate / 12
n = 60

#Monthly Payment Formula
M = P * r * (1 + r) * n / ((1 + r) * n - 1)
print(f"Fixed Monthly Payment: {M:,.2f}\n")

balance = P

print(f"{'Month':<6}{'Payment':>12}{'Interest':>12}{'Principal':>12}{'Balance':>15}")
for month in range(1, n + 1):
    interest = balance * r
    principal = M - interest
    balance -= principal
    print(f"{month:<6}{M:>12,.2f}{interest:>12,.2f}{principal:>12,.2f}{balance:>15,.2f}")

Fixed Monthly Payment: 508,389.26

Month      Payment    Interest   Principal        Balance
1       508,389.26  500,000.00    8,389.26  49,991,610.74
2       508,389.26  499,916.11    8,473.15  49,983,137.58
3       508,389.26  499,831.38    8,557.89  49,974,579.70
4       508,389.26  499,745.80    8,643.46  49,965,936.23
5       508,389.26  499,659.36    8,729.90  49,957,206.33
6       508,389.26  499,572.06    8,817.20  49,948,389.14
7       508,389.26  499,483.89    8,905.37  49,939,483.77
8       508,389.26  499,394.84    8,994.42  49,930,489.34
9       508,389.26  499,304.89    9,084.37  49,921,404.97
10      508,389.26  499,214.05    9,175.21  49,912,229.76
11      508,389.26  499,122.30    9,266.96  49,902,962.80
12      508,389.26  499,029.63    9,359.63  49,893,603.16
13      508,389.26  498,936.03    9,453.23  49,884,149.93
14      508,389.26  498,841.50    9,547.76  49,874,602.17
15      508,389.26  498,746.02    9,643.24  49,864,958.93
16      508,389.26  498,649.59    9,7

In [95]:
import csv
from datetime import datetime, timedelta

def find_planting_window(weather_data, year):
    """Finds the first 7-day planting window in a year meeting temp and rainfall criteria."""
    yearly_data = sorted([d for d in weather_data if d['Date'].year == year], key=lambda x: x['Date'])
    
    for i in range(len(yearly_data) - 6):
        window = yearly_data[i : i + 7]
        
        # Ensure consecutive dates
        if any((window[j+1]['Date'] - window[j]['Date']).days != 1 for j in range(6)):
            continue

        avg_temp = sum((d['MinTemp'] + d['MaxTemp']) / 2 for d in window) / 7
        total_rainfall = sum(d['Rainfall_mm'] for d in window)

        if avg_temp > 18 and total_rainfall >= 25:
            return window[0]['Date']
    return None

def main():
    csv_filename = 'morogoro_weather.csv'
    # Create dummy CSV (simplified for brevity, actual data generation is in long form)
    with open(csv_filename, 'w', newline='') as f:
        f.write("Date,MinTemp,MaxTemp,Rainfall_mm\n")
        # Add varied data for 2015-2024 to simulate different planting windows
        # Example for a few relevant days:
        f.write("2020-04-09,20.5,28.0,4.0\n2020-04-10,21.0,29.0,5.0\n2020-04-11,20.8,28.5,6.0\n2020-04-12,21.5,29.5,4.5\n2020-04-13,20.0,27.0,3.0\n2020-04-14,21.2,28.8,3.5\n2020-04-15,20.7,28.2,5.0\n") # Total 31mm, avg temp > 18
        f.write("2023-05-20,20.1,28.0,5.0\n2023-05-21,20.5,28.5,4.0\n2023-05-22,20.3,28.2,3.5\n2023-05-23,21.0,29.0,6.0\n2023-05-24,20.8,28.7,4.0\n2023-05-25,20.2,28.1,3.0\n2023-05-26,20.6,28.3,2.0\n") # Total 27.5mm, avg temp > 18
        f.write("2017-04-30,20.0,28.0,5.0\n2017-05-01,20.5,28.5,4.0\n2017-05-02,20.3,28.2,3.5\n2017-05-03,21.0,29.0,6.0\n2017-05-04,20.8,28.7,4.0\n2017-05-05,20.2,28.1,3.0\n2017-05-06,20.6,28.3,2.0\n") # Total 27.5mm, avg temp > 18
        # Add other years with various or no valid windows to make it realistic
        for year in range(2015, 2025):
            if year not in [2017, 2020, 2023]: # Add some generic data for other years
                for day in range(120, 150): # Cover some days
                    current_date = datetime(year, 1, 1) + timedelta(days=day - 1)
                    f.write(f"{current_date.strftime('%Y-%m-%d')},{random.uniform(15,22):.1f},{random.uniform(25,32):.1f},{random.uniform(0,10):.1f}\n")


    weather_data = []
    with open(csv_filename, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                weather_data.append({
                    'Date': datetime.strptime(row['Date'], '%Y-%m-%d'),
                    'MinTemp': float(row['MinTemp']),
                    'MaxTemp': float(row['MaxTemp']),
                    'Rainfall_mm': float(row['Rainfall_mm'])
                })
            except (ValueError, KeyError) as e: pass # Skip bad rows

    planting_dates = {}
    for year in sorted(list(set(d['Date'].year for d in weather_data))):
        start_date = find_planting_window(weather_data, year)
        if start_date:
            planting_dates[year] = start_date
            print(f"Year {year}: {start_date.strftime('%Y-%m-%d')} (Day {start_date.timetuple().tm_yday})")
        else:
            print(f"Year {year}: No valid window.")

    if not planting_dates: print("\nNo windows found."); return

    avg_day = sum(d.timetuple().tm_yday for d in planting_dates.values()) / len(planting_dates)
    avg_date = (datetime(2023, 1, 1) + timedelta(days=int(avg_day) - 1)).strftime('%B %dth')
    print(f"\nOptimal planting: Day {int(avg_day)} ({avg_date}).")

    earliest_year = min(planting_dates, key=planting_dates.get)
    latest_year = max(planting_dates, key=planting_dates.get)
    print(f"Earliest: {planting_dates[earliest_year].strftime('%Y-%m-%d')} ({earliest_year})")
    print(f"Latest: {planting_dates[latest_year].strftime('%Y-%m-%d')} ({latest_year})")

if __name__ == "__main__":
    import random # Only needed for dummy data generation
    main()

Year 2015: 2015-05-03 (Day 123)
Year 2016: 2016-04-29 (Day 120)
Year 2017: 2017-04-30 (Day 120)
Year 2018: 2018-04-30 (Day 120)
Year 2019: 2019-05-01 (Day 121)
Year 2020: 2020-04-09 (Day 100)
Year 2021: 2021-05-01 (Day 121)
Year 2022: 2022-04-30 (Day 120)
Year 2023: 2023-05-20 (Day 140)
Year 2024: 2024-04-29 (Day 120)

Optimal planting: Day 120 (April 30th).
Earliest: 2015-05-03 (2015)
Latest: 2024-04-29 (2024)


In [97]:

import statistics

#Step 1: Input yield data
treatments = ['T1', 'T2', 'T3', 'C']
data = {}

print(" Enter yield (tonnes/ha) for each treatment and block:")
for trt in treatments:
    data[trt] = []
    for block in range(1, 5):
        val = float(input(f"Enter yield for Treatment {trt}, Block {block}: "))
        data[trt].append(val)

#Step 2: Calculate stats
results = {}
for trt, yields in data.items():
    mean = statistics.mean(yields)
    std = statistics.stdev(yields)
    cv = (std / mean) * 100  # Coefficient of Variation (%)
    results[trt] = {
        "Mean": mean,
        "SD": std,
        "CV": cv
    }

#Step 3: Print Summary Table
print("\n Summary Report")
print(f"{'Treatment':<10}{'Mean':>10}{'SD':>10}{'CV (%)':>10}")
for trt, stats in results.items():
    print(f"{trt:<10}{stats['Mean']:>10.2f}{stats['SD']:>10.2f}{stats['CV']:>10.2f}")

#Step 4: Identify best treatments
best_yield = max(results.items(), key=lambda x: x[1]['Mean'])[0]
best_cv = min(results.items(), key=lambda x: x[1]['CV'])[0]

print(f"\n Highest Mean Yield: {best_yield}")
print(f" Most Consistent (Lowest CV): {best_cv}")




 Enter yield (tonnes/ha) for each treatment and block:


Enter yield for Treatment T1, Block 1:  3.2
Enter yield for Treatment T1, Block 2:  3.5
Enter yield for Treatment T1, Block 3:  3.4
Enter yield for Treatment T1, Block 4:  3.3
Enter yield for Treatment T2, Block 1:  4.1
Enter yield for Treatment T2, Block 2:  4.0
Enter yield for Treatment T2, Block 3:  4.3
Enter yield for Treatment T2, Block 4:  4.2
Enter yield for Treatment T3, Block 1:  2.8
Enter yield for Treatment T3, Block 2:  2.9
Enter yield for Treatment T3, Block 3:  3.0
Enter yield for Treatment T3, Block 4:  2.7
Enter yield for Treatment C, Block 1:  2.5
Enter yield for Treatment C, Block 2:  2.6
Enter yield for Treatment C, Block 3:  2.4
Enter yield for Treatment C, Block 4:  2.5



 Summary Report
Treatment       Mean        SD    CV (%)
T1              3.35      0.13      3.85
T2              4.15      0.13      3.11
T3              2.85      0.13      4.53
C               2.50      0.08      3.27

 Highest Mean Yield: T2
 Most Consistent (Lowest CV): T2


In [101]:
# Define the Farm class
class Farm:
    def __init__(self, owner_name, monthly_allocation):
        self.owner_name = owner_name
        self.monthly_allocation = monthly_allocation  # Total allowed water in a month
        self.water_used_this_month = 0  # Starts at zero at the beginning of the month

    def available_water(self):
        # Returns the remaining water the farm can still use this month
        return self.monthly_allocation - self.water_used_this_month

# Define the WaterAuthority class
class WaterAuthority:
    def __init__(self):
        self.farms = {}  # A dictionary to store Farm objects by their owner's name

    def add_farm(self, farm):
        self.farms[farm.owner_name] = farm  # Add farm to the dictionary

    def request_water(self, farm_name, amount):
        if farm_name in self.farms:
            farm = self.farms[farm_name]
            if amount <= farm.available_water():
                farm.water_used_this_month += amount
                print(f"Water request approved for {farm_name}: {amount} units used.")
            else:
                print(f"Water request denied for {farm_name}: Not enough allocation.")
        else:
            print(f"Farm {farm_name} not found.")

    def trade_water(self, from_farm_name, to_farm_name, amount):
        if from_farm_name in self.farms and to_farm_name in self.farms:
            seller = self.farms[from_farm_name]
            buyer = self.farms[to_farm_name]

            if amount <= seller.available_water():
                # Atomic trade: update both allocations
                seller.monthly_allocation -= amount
                buyer.monthly_allocation += amount
                print(f"{from_farm_name} traded {amount} units to {to_farm_name}.")
            else:
                print(f"Trade failed: {from_farm_name} does not have enough surplus.")
        else:
            print("One or both farms not found.")

    def reset_month(self):
        # Reset water usage at the start of a new month
        for farm in self.farms.values():
            farm.water_used_this_month = 0

    def report(self):
        # Print current status of all farms
        print("\n--- Monthly Report ---")
        for farm in self.farms.values():
            print(f"{farm.owner_name}: Used {farm.water_used_this_month}/{farm.monthly_allocation} units")

# Simulation of a month
authority = WaterAuthority()

# Adding farms to the authority
authority.add_farm(Farm("Alice", 1000))
authority.add_farm(Farm("Bob", 800))
authority.add_farm(Farm("Carlos", 1200))

# Simulate water requests
authority.request_water("Alice", 300)
authority.request_water("Bob", 500)
authority.request_water("Carlos", 1100)
authority.request_water("Bob", 400)  # Should be denied

# Trade water from Carlos to Bob
authority.trade_water("Carlos", "Bob", 100)

# Bob requests more water after trade
authority.request_water("Bob", 200)  # Should now be approved

# Print monthly usage report
authority.report()

Water request approved for Alice: 300 units used.
Water request approved for Bob: 500 units used.
Water request approved for Carlos: 1100 units used.
Water request denied for Bob: Not enough allocation.
Carlos traded 100 units to Bob.
Water request approved for Bob: 200 units used.

--- Monthly Report ---
Alice: Used 300/1000 units
Bob: Used 700/900 units
Carlos: Used 1100/1100 units


In [103]:
class Population:
    def __init__(self, count, rate):
        self.count = count
        self.growth_rate = rate

def run_ipm_tool():
    print("--- Fall Armyworm IPM Tool ---")
    PREDATOR_EFFICIENCY = 5 # 1 predator eliminates 5 pests
    RISK_LOW = 0.25
    RISK_HIGH = 0.6

    for week in range(1, 9):
        print(f"\n--- Week {week} ---")
        while True:
            try:
                pest_c = float(input("Enter scouted Fall Armyworm count: "))
                if pest_c >= 0: break
                else: print("Count cannot be negative.")
            except ValueError: print("Invalid number.")
        
        while True:
            try:
                pred_c = float(input("Enter scouted beneficial predator count: "))
                if pred_c >= 0: break
                else: print("Count cannot be negative.")
            except ValueError: print("Invalid number.")

        if pest_c == 0:
            risk = 0.0
        else:
            risk = (pest_c - (pred_c * PREDATOR_EFFICIENCY)) / pest_c
        
        print(f"Risk Index: {risk:.2f}")

        if risk < RISK_LOW:
            print("Recommendation: Infestation low. No action needed.")
        elif RISK_LOW <= risk < RISK_HIGH:
            print("Recommendation: Warning: Pest pressure increasing. Consider releasing more beneficial insects.")
        else:
            print("Recommendation: ALERT: Economic threshold crossed. Chemical intervention may be required.")
        
        if week < 8: input("\nPress Enter for next week...")
    
    print("\n--- End of Season ---")

if __name__ == "__main__":
    run_ipm_tool()

--- Fall Armyworm IPM Tool ---

--- Week 1 ---


Enter scouted Fall Armyworm count:  50
Enter scouted beneficial predator count:  20


Risk Index: -1.00
Recommendation: Infestation low. No action needed.



Press Enter for next week... 



--- Week 2 ---


...

Risk Index: 0.00
Recommendation: Infestation low. No action needed.



Press Enter for next week... 



--- Week 8 ---


Enter scouted Fall Armyworm count:  30
Enter scouted beneficial predator count:  10


Risk Index: -0.67
Recommendation: Infestation low. No action needed.

--- End of Season ---
